# Simple example on use of PYOMO for a LP problem.

The company Paint Deals produces two colors of paint, blue and black.
- Blue paint is sold for US\\$10 per liter, while black paint is sold for US\\$15
per liter.
- The company owns a process plant which can produce one color paint at
a time.
- However, blue paint is produced at a rate of 40 liters per hour, while the
production rate for black paint is 30 liters per hour.
- Besides, the marketing department estimates that at most 860 liters of
black paint and 1000 liters of blue paint can be sold in the market.
- During a week, the plant can operate for 40 hours and the paint can be
stored for the following week.
- Determine how many liters of each paint should be produced to maximize
week revenue.

This problem can be written as a linear program:
\begin{align}
\max \quad & 10 \cdot \text{BluePaint} + 15 \cdot \text{BlackPaint} \\
 \text{subject to} \quad & \frac{1}{40} \cdot \text{BluePaint} + \frac{1}{30} \cdot \text{BlackPaint} \leq 40 \\
 & 0 \leq \text{BluePaint} \leq 860 \\
 & 0 \leq \text{BlackPaint} \leq 1000 
\end{align}

We start by setting up PYOMO to solve a 'Concrete' model:

In [24]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# create model
m = ConcreteModel()

Then we define our variables:

In [25]:
m.BluePaint = Var(domain=NonNegativeReals)
m.BlackPaint = Var(domain=NonNegativeReals)

Note that by specifying the domain as NonNegativeReals, we have already implemented the lower bound on the variables.

Next, we specify the objective function. Note that Pyomo by default assumes minimization problems, therefore we must specify that this is a maximization problem.

In [26]:
m.obj = Objective( expr = 10*m.BluePaint + 15*m.BlackPaint, sense=maximize)

Then, we specify the constraints:

In [27]:
m.Constraint1 = Constraint(expr = (1/40)*m.BluePaint + (1/30)*m.BlackPaint <= 40)
m.Constraint2 = Constraint(expr = m.BluePaint <= 860)
m.Constraint3 = Constraint(expr = m.BlackPaint <= 1000)

Now, we are ready to solve. We will use the open source solver GLPK:

In [28]:
opt = SolverFactory('glpk')
results = opt.solve(m)

Let us have a look at the solver output:

In [29]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17666.6666666667
  Upper bound: 17666.6666666667
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.017953157424926758
# ----------------------------------------------------------
#   Solution Information
# --------------

For some reason, it reports an extra variable.

When Pyomo solves, it copies the solution into the problem instant. Therefore, the solution can be inspected by looking at the values of the variables in the model.

In [30]:
m.BluePaint.display()
m.BlackPaint.display()

BluePaint : Size=1, Index=None
    Key  : Lower : Value            : Upper : Fixed : Stale : Domain
    None :     0 : 266.666666666667 :  None : False : False : NonNegativeReals
BlackPaint : Size=1, Index=None
    Key  : Lower : Value  : Upper : Fixed : Stale : Domain
    None :     0 : 1000.0 :  None : False : False : NonNegativeReals


More commonly, we use vectors when we specify and solve optimization problems. The code for the above problem using vectors are given below:

In [34]:
# Create a new model
m2 = ConcreteModel()
# Create a vector of two (nonnegative) variables
m2.x = Var([1,2], domain=NonNegativeReals)
# Objective function
m2.obj = Objective(expr = -10*m2.x[1] - 15*m2.x[2])
# Constraints, here using ConstraintList
m2.Constraints = ConstraintList()
m2.Constraints.add(expr = (1/40)*m2.x[1] + (1/30)*m2.x[2] <= 40)
m2.Constraints.add(expr = m2.x[1] <= 860)
m2.Constraints.add(expr = m2.x[2] <= 1000)

results = opt.solve(m2)

In [35]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -17666.6666666667
  Upper bound: -17666.6666666667
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 5
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01595759391784668
# ----------------------------------------------------------
#   Solution Information
# -------------

In [36]:
m2.x.display()

x : Size=2, Index=x_index
    Key : Lower : Value            : Upper : Fixed : Stale : Domain
      1 :     0 : 266.666666666667 :  None : False : False : NonNegativeReals
      2 :     0 :           1000.0 :  None : False : False : NonNegativeReals
